Import Statements

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import pandas as pd
import numpy as np
from datetime import datetime
import lightgbm as lgb
from dotenv import load_dotenv
load_dotenv()
from sklearn.metrics import mean_absolute_error

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from src.config import TRANSFORMED_DATA_DIR
from src.data_utils import split_time_series_data
from src.experiment_utils import set_mlflow_tracking, log_model_to_mlflow

Data Loading and Test Train split

In [2]:
# Load the tabular data
df = pd.read_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet")
df.head(5)

# Split the data into training and testing sets
# Training period: January 2024 to August 2024
# Test period: September 2024 to January 2025
X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2025, 1, 1, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# Select only the numeric features (lagged ride counts)
past_ride_columns = [c for c in X_train.columns if c.startswith("rides_")]
X_train_only_numeric = X_train[past_ride_columns]
X_test_only_numeric = X_test[past_ride_columns]

(24336, 674)
(24336,)
(2232, 674)
(2232,)


LightGBM Model Predictions and Logging

In [3]:
# Train a LightGBM model
model = lgb.LGBMRegressor()
model.fit(X_train_only_numeric, y_train)

# Make predictions and evaluate
predictions = model.predict(X_test_only_numeric)
test_mae = mean_absolute_error(y_test, predictions)
print(f"LightGBM MAE: {test_mae:.4f}")

# Log the model to MLflow
mlflow = set_mlflow_tracking()
log_model_to_mlflow(model, X_test_only_numeric, "LGBMRegressor", "mean_absolute_error", score=test_mae)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.043955 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 65856
[LightGBM] [Info] Number of data points in the train set: 24336, number of used features: 672
[LightGBM] [Info] Start training from score 17.580950


INFO:src.experiment_utils:MLflow tracking URI and credentials set.


LightGBM MAE: 3.0292


INFO:src.experiment_utils:Experiment set to: LGBMRegressor
INFO:src.experiment_utils:Logged mean_absolute_error: 3.0291538263036446
c:\Users\singh\Downloads\CDS500_Applied_ML_DS\Projects\CDA500Final\CDA500PF2\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
INFO:src.experiment_utils:Model signature inferred.


2025/05/01 00:03:46 INFO mlflow.models.model: Found the following environment variables used during model inference: [HOPSWORKS_API_KEY]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
Registered model 'LGBMRegressor' already exists. Creating a new version of this model...
2025/05/01 00:04:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LGBMRegressor, version 5
Created version '5' of model 'LGBMRegressor'.
INFO:src.experiment_utils:Model logged with name: LGBMRegressor


🏃 View run intrigued-rat-670 at: https://dagshub.com/singhvarun0405/CDA500PF1.mlflow/#/experiments/4/runs/8e625681763342afa9428a449dbe07fc
🧪 View experiment at: https://dagshub.com/singhvarun0405/CDA500PF1.mlflow/#/experiments/4


In [4]:
# Compute Safe MAPE (Mean Absolute Percentage Error)
def safe_mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Only compute MAPE for non-zero actual values to avoid division by zero issues
    non_zero_mask = y_true != 0
    if not np.any(non_zero_mask):
        return np.nan  # Return NaN if all actual values are 0
    y_true_non_zero = y_true[non_zero_mask]
    y_pred_non_zero = y_pred[non_zero_mask]
    # Add a small epsilon to the denominator for numerical stability
    epsilon = 1e-2  # Larger epsilon to prevent extreme values
    mape = np.mean(np.abs((y_true_non_zero - y_pred_non_zero) / (y_true_non_zero + epsilon))) * 100
    return mape

mape = safe_mean_absolute_percentage_error(y_test, predictions)
print(f"LightGBM Safe MAPE (non-zero actuals): {mape:.4f}%")

LightGBM Safe MAPE (non-zero actuals): 48.1654%
